In [3]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import gzip
import os
import pickle
import json as json

import numpy as np
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.INFO)

In [3]:
class JSONEnc(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(JSONEnc, self).default(obj)

def one_hot(labels):
    """this creates a one hot encoding from a flat vector:
    i.e. given y = [0,2,1]
     it creates y_one_hot = [[1,0,0], [0,0,1], [0,1,0]]
    """
    classes = np.unique(labels)
    n_classes = classes.size
    one_hot_labels = np.zeros(labels.shape + (n_classes,))
    for c in classes:
        one_hot_labels[labels == c, c] = 1
    return one_hot_labels

def unhot(one_hot_labels):
    """ Invert a one hot encoding, creating a flat vector """
    return np.argmax(one_hot_labels, axis=-1)

def mnist(datasets_dir='./data'):
    if not os.path.exists(datasets_dir):
        os.mkdir(datasets_dir)
    data_file = os.path.join(datasets_dir, 'mnist.pkl.gz')
    if not os.path.exists(data_file):
        print('... downloading MNIST from the web')
        try:
            import urllib
            urllib.urlretrieve('http://google.com')
        except AttributeError:
            import urllib.request as urllib
        url = 'http://www.iro.umontreal.ca/~lisa/deep/data/mnist/mnist.pkl.gz'
        urllib.urlretrieve(url, data_file)

    print('... loading data')
    # Load the dataset
    f = gzip.open(data_file, 'rb')
    try:
        train_set, valid_set, test_set = pickle.load(f, encoding="latin1")
    except TypeError:
        train_set, valid_set, test_set = pickle.load(f)
    f.close()

    test_x, test_y = test_set
    test_x = test_x.astype('float32')
    test_x = test_x.astype('float32').reshape(test_x.shape[0], 28, 28, 1)
    test_y = test_y.astype('int32')
    valid_x, valid_y = valid_set
    valid_x = valid_x.astype('float32')
    valid_x = valid_x.astype('float32').reshape(valid_x.shape[0], 28, 28, 1)
    valid_y = valid_y.astype('int32')
    train_x, train_y = train_set
    train_x = train_x.astype('float32').reshape(train_x.shape[0], 28, 28, 1)
    train_y = train_y.astype('int32')
    print('... done loading data')
#     return train_x, one_hot(train_y), valid_x, one_hot(valid_y), test_x, one_hot(test_y)
    return train_x, train_y, valid_x, valid_y, test_x, test_y

# Using tf Estimators
# Conv Net Function
def build_cnn_model(features, labels, mode, params):
    # STEP 1 : BUILD CONVNET
    # GRAPH : I/P --> CL1 --> RELU --> POOL1 --> CL2 --> RELU --> POOL2 --> FC --> SOFTMAX
    
    # Hyperparameters
    lr = params['lr']
    
    num_filters = params['num_filters']
    
    filter_size = params['filter_size']
    
    x_input = tf.reshape(features["x_ip"], [-1, 28, 28, 1])
    convlayer1 = tf.layers.conv2d(inputs=x_input, filters=num_filters, kernel_size=filter_size,\
                                  padding="same", activation=tf.nn.relu)

    maxpool1 = tf.layers.max_pooling2d(inputs=convlayer1, pool_size=2, strides=2)
    
    convlayer2 = tf.layers.conv2d(inputs=maxpool1, filters=num_filters, kernel_size=filter_size,\
                                  padding="same", activation=tf.nn.relu)
    
    maxpool2 = tf.layers.max_pooling2d(inputs=convlayer2, pool_size=2, strides=2)
    
    maxpool2_vec = tf.reshape(maxpool2, [-1, maxpool2.shape[1] * maxpool2.shape[2] * num_filters])
    
    denselayer = tf.layers.dense(inputs=maxpool2_vec, units=128)
    
    # Logits Layer
    logitslayer = tf.layers.dense(inputs=denselayer, units=10, activation=None)

    # Generate predictions (for PREDICT and EVAL mode)
    predictions = { "classes": tf.argmax(input=logitslayer, axis=1),\
                   "probabilities": tf.nn.softmax(logitslayer, name="softmax_tensor") }
    
    # Calculate Loss (for both TRAIN and EVAL modes)
    sceloss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logitslayer)
    
    # Generate estimator object for TRAIN
    if mode == tf.estimator.ModeKeys.TRAIN:
        print("tf.estimator.ModeKeys.TRAIN...")
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=lr)
        train_op = optimizer.minimize(loss=sceloss,global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(tf.estimator.ModeKeys.TRAIN,loss=sceloss, train_op=train_op)
    
    # Add evaluation metrics (for EVAL mode)
    if mode == tf.estimator.ModeKeys.EVAL: 
        print("tf.estimator.ModeKeys.EVAL...")
        eval_metric_ops = { "accuracy": tf.metrics.accuracy(labels=labels, predictions=predictions["classes"])}
        return tf.estimator.EstimatorSpec(mode=mode, loss=sceloss, eval_metric_ops=eval_metric_ops)

def train_and_validate(x_train, y_train, x_valid, y_valid, num_epochs, lr, num_filters, batch_size, filter_size):
    # TODO: train and validate your convolutional neural networks with the provided data and hyperparameters
    learning_curve = []
    param_dict = {'lr': lr, 'num_filters': num_filters, 'filter_size': filter_size,}
    dir_name = "./mnist_convnet_modellearningrate_" + str(lr)
    print(dir_name)
    
    # Logging Hook
    tensors_to_log = {"probabilities": "softmax_tensor"}
    log_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=10)
    
    # Create the Estimator
    MNIST_estimator = tf.estimator.Estimator(model_fn=build_cnn_model, params=param_dict, model_dir=dir_name)
    
    # Training Param Func
    train_func = tf.estimator.inputs.numpy_input_fn(x={"x_ip": x_train}, y=y_train, batch_size=batch_size,\
                                                    num_epochs=None, shuffle=True)
    # Val Param Func
    val_func = tf.estimator.inputs.numpy_input_fn(x={"x_ip": x_valid}, y=y_valid, num_epochs=1, shuffle=False)
    
    # Train & Validate the model/estimator
    for epoch in range(1, num_epochs + 1):
        te = MNIST_estimator.train(input_fn=train_func, hooks=[log_hook], steps=1)
        print("TRAINING EPOCH...", epoch)
        print(te)

        ve = MNIST_estimator.evaluate(input_fn=val_func)
        print("EVAL EPOCH...", epoch)
        print(ve)
        learning_curve.append(ve)
    
    return learning_curve, MNIST_estimator  # TODO: Return the validation error after each epoch (i.e learning curve) and your model


def test(x_test, y_test, model):
    # TODO: test your network here by evaluating it on the test data
    test_func = tf.estimator.inputs.numpy_input_fn(x={"x_ip": x_test}, y=y_test, num_epochs=1, shuffle=False)
    test_err = model.evaluate(input_fn=test_func)
    return test_err


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--output_path", default="./", type=str, nargs="?",
                        help="Path where the results will be stored")
    parser.add_argument("--input_path", default="./", type=str, nargs="?",
                        help="Path where the data is located. If the data is not available it will be downloaded first")
    parser.add_argument("--learning_rate", default=1e-3, type=float, nargs="?", help="Learning rate for SGD")
    parser.add_argument("--num_filters", default=32, type=int, nargs="?",
                        help="The number of filters for each convolution layer")
    parser.add_argument("--batch_size", default=128, type=int, nargs="?", help="Batch size for SGD")
    parser.add_argument("--epochs", default=12, type=int, nargs="?",
                        help="Determines how many epochs the network will be trained")
    parser.add_argument("--run_id", default=0, type=int, nargs="?",
                        help="Helps to identify different runs of an experiments")
    parser.add_argument("--filter_size", default=3, type=int, nargs="?",
                        help="Filter width and height")


    base_path = "./"
    
    # hyperparameters
    lrs = [0.1, 0.01, 0.001, 0.0001] #args.learning_rate
    lr = lrs[0]
    num_filters = 16 
    batch_size = 128 
    epochs = 500 
    filter_size = 3 

    x_train, y_train, x_valid, y_valid, x_test, y_test = mnist(base_path)
        
    learning_curve, model = train_and_validate(x_train, y_train, x_valid, y_valid, epochs, lr, num_filters, \
                                               batch_size, filter_size)
    test_error = test(x_test, y_test, model)

    # save results in a dictionary and write them into a .json file
    results = dict()
    results["lr"] = lr
    results["num_filters"] = num_filters
    results["batch_size"] = batch_size
    results["filter_size"] = filter_size
    results["learning_curve"] = learning_curve
    results["test_error"] = test_error

    print(results)

    path = os.path.join(base_path, "results")
    os.makedirs(path, exist_ok=True)
    
    fname = os.path.join(path, "results_run_lr_%2f.json" % lr)
    fh = open(fname, "w")
    json.dump(results, fh, cls=JSONEnc)
    fh.close()

... loading data
... done loading data
(50000, 28, 28, 1)
(50000,)
(10000, 28, 28, 1)
(10000,)
(10000, 28, 28, 1)
(10000,)
./mnist_convnet_modellearningrate_0.1
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './mnist_convnet_modellearningrate_0.1', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000000000B53F518>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num

INFO:tensorflow:loss = 2.2579348, step = 3
INFO:tensorflow:Saving checkpoints for 3 into ./mnist_convnet_modellearningrate_0.1\model.ckpt.
INFO:tensorflow:Loss for final step: 2.2579348.
TRAINING EPOCH... 3
INFO:tensorflow:Calling model_fn.
x_input.shape (?, 28, 28, 1)
convlayer1.shape (?, 28, 28, 16)
maxpool1.shape (?, 14, 14, 16)
convlayer2.shape (?, 14, 14, 16)
maxpool2.shape (?, 7, 7, 16)
maxpool2_vec.shape (?, 784)
denselayer.shape (?, 128)
logitslayer.shape (?, 10)
labels.shape (?,)
sceloss.shape ()
tf.estimator.ModeKeys.EVAL...
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-11-13-20:49:43
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./mnist_convnet_modellearningrate_0.1\model.ckpt-3
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-11-13-20:49:45
INFO:tensorflow:Saving dict for global step 3: accuracy = 0.2052, global_step = 3, loss =

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./mnist_convnet_modellearningrate_0.1\model.ckpt-6
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-11-13-20:50:04
INFO:tensorflow:Saving dict for global step 6: accuracy = 0.3435, global_step = 6, loss = 2.1737266
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 6: ./mnist_convnet_modellearningrate_0.1\model.ckpt-6
EVAL EPOCH... 6
{'accuracy': 0.3435, 'loss': 2.1737266, 'global_step': 6}
INFO:tensorflow:Calling model_fn.
x_input.shape (128, 28, 28, 1)
convlayer1.shape (128, 28, 28, 16)
maxpool1.shape (128, 14, 14, 16)
convlayer2.shape (128, 14, 14, 16)
maxpool2.shape (128, 7, 7, 16)
maxpool2_vec.shape (128, 784)
denselayer.shape (128, 128)
logitslayer.shape (128, 10)
labels.shape (128,)
sceloss.shape ()
tf.estimator.ModeKeys.TRAIN...
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./mnist_convnet_modellearningrate_0.1\model.ckpt-9
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 9 into ./mnist_convnet_modellearningrate_0.1\model.ckpt.
INFO:tensorflow:probabilities = [[0.10884596 0.09016409 0.1465489  ... 0.07022099 0.12402266 0.07290816]
 [0.19009386 0.05770925 0.08086592 ... 0.06894704 0.15113908 0.07285581]
 [0.1311565  0.07027584 0.10451129 ... 0.07784545 0.09948357 0.06662207]
 ...
 [0.11696094 0.07405686 0.12569536 ... 0.09781895 0.13367158 0.09457558]
 [0.09214608 0.09217036 0.11567151 ... 0.11271605 0.10918812 0.10196821]
 [0.10235757 0.09744295 0.11997894 ... 0.11229236 0.13588376 0.09587976]]
INFO:tensorflow:loss = 2.0590987, step = 10
INFO:tensorflow:Saving checkpoints for 10 into ./mnist_convnet_modellearningrate_0.1\model.ckpt.
INFO:tensorflow:Loss for final

INFO:tensorflow:loss = 1.9031398, step = 13
INFO:tensorflow:Saving checkpoints for 13 into ./mnist_convnet_modellearningrate_0.1\model.ckpt.
INFO:tensorflow:Loss for final step: 1.9031398.
TRAINING EPOCH... 13
INFO:tensorflow:Calling model_fn.
x_input.shape (?, 28, 28, 1)
convlayer1.shape (?, 28, 28, 16)
maxpool1.shape (?, 14, 14, 16)
convlayer2.shape (?, 14, 14, 16)
maxpool2.shape (?, 7, 7, 16)
maxpool2_vec.shape (?, 784)
denselayer.shape (?, 128)
logitslayer.shape (?, 10)
labels.shape (?,)
sceloss.shape ()
tf.estimator.ModeKeys.EVAL...
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-11-13-20:50:45
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./mnist_convnet_modellearningrate_0.1\model.ckpt-13
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-11-13-20:50:47
INFO:tensorflow:Saving dict for global step 13: accuracy = 0.5595, global_step = 13, 

...

INFO:tensorflow:Starting evaluation at 2018-11-13-22:16:29
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./mnist_convnet_modellearningrate_0.1\model.ckpt-485
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-11-13-22:16:31
INFO:tensorflow:Saving dict for global step 485: accuracy = 0.9657, global_step = 485, loss = 0.11709166
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 485: ./mnist_convnet_modellearningrate_0.1\model.ckpt-485
EVAL EPOCH... 485
{'accuracy': 0.9657, 'loss': 0.11709166, 'global_step': 485}
INFO:tensorflow:Calling model_fn.
x_input.shape (128, 28, 28, 1)
convlayer1.shape (128, 28, 28, 16)
maxpool1.shape (128, 14, 14, 16)
convlayer2.shape (128, 14, 14, 16)
maxpool2.shape (128, 7, 7, 16)
maxpool2_vec.shape (128, 784)
denselayer.shape (128, 128)
logitslayer.shape (128, 10)
labels.shape (128,)
sceloss.shape ()
tf.estimator.ModeKeys.TRAIN...
INFO:te

x_input.shape (128, 28, 28, 1)
convlayer1.shape (128, 28, 28, 16)
maxpool1.shape (128, 14, 14, 16)
convlayer2.shape (128, 14, 14, 16)
maxpool2.shape (128, 7, 7, 16)
maxpool2_vec.shape (128, 784)
denselayer.shape (128, 128)
logitslayer.shape (128, 10)
labels.shape (128,)
sceloss.shape ()
tf.estimator.ModeKeys.TRAIN...
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./mnist_convnet_modellearningrate_0.1\model.ckpt-488
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 488 into ./mnist_convnet_modellearningrate_0.1\model.ckpt.
INFO:tensorflow:probabilities = [[0.00000048 0.9986523  0.00004331 ... 0.00005876 0.00060786 0.00003068]
 [0.0000005  0.00000092 0.00008801 ... 0.00002397 0.00001104 0.00000003]
 [0.00000008 0.99950004 0.00007142 ... 0.00000455 0.00022744 0.00000286]
 ...
 [0.00000021 0.         0.000

INFO:tensorflow:Saving checkpoints for 491 into ./mnist_convnet_modellearningrate_0.1\model.ckpt.
INFO:tensorflow:probabilities = [[0.99985564 0.         0.00001071 ... 0.         0.00001859 0.00000025]
 [0.00006277 0.0000015  0.00321421 ... 0.00124366 0.7845029  0.20981936]
 [0.         0.         0.         ... 0.9999528  0.00000001 0.00004726]
 ...
 [0.9999864  0.         0.00000006 ... 0.00000005 0.00000002 0.00000027]
 [0.00000135 0.00000002 0.00000653 ... 0.         0.00000361 0.        ]
 [0.9984106  0.         0.00002074 ... 0.00000229 0.00003995 0.00008916]]
INFO:tensorflow:loss = 0.035505068, step = 492
INFO:tensorflow:Saving checkpoints for 492 into ./mnist_convnet_modellearningrate_0.1\model.ckpt.
INFO:tensorflow:Loss for final step: 0.035505068.
TRAINING EPOCH... 492
INFO:tensorflow:Calling model_fn.
x_input.shape (?, 28, 28, 1)
convlayer1.shape (?, 28, 28, 16)
maxpool1.shape (?, 14, 14, 16)
convlayer2.shape (?, 14, 14, 16)
maxpool2.shape (?, 7, 7, 16)
maxpool2_vec.shape (

INFO:tensorflow:Saving checkpoints for 495 into ./mnist_convnet_modellearningrate_0.1\model.ckpt.
INFO:tensorflow:Loss for final step: 0.13257079.
TRAINING EPOCH... 495
INFO:tensorflow:Calling model_fn.
x_input.shape (?, 28, 28, 1)
convlayer1.shape (?, 28, 28, 16)
maxpool1.shape (?, 14, 14, 16)
convlayer2.shape (?, 14, 14, 16)
maxpool2.shape (?, 7, 7, 16)
maxpool2_vec.shape (?, 784)
denselayer.shape (?, 128)
logitslayer.shape (?, 10)
labels.shape (?,)
sceloss.shape ()
tf.estimator.ModeKeys.EVAL...
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-11-13-22:17:19
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./mnist_convnet_modellearningrate_0.1\model.ckpt-495
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-11-13-22:17:21
INFO:tensorflow:Saving dict for global step 495: accuracy = 0.9601, global_step = 495, loss = 0.13331707
INFO:tensorflow:Savi

INFO:tensorflow:Starting evaluation at 2018-11-13-22:17:35
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./mnist_convnet_modellearningrate_0.1\model.ckpt-498
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-11-13-22:17:37
INFO:tensorflow:Saving dict for global step 498: accuracy = 0.9712, global_step = 498, loss = 0.10519544
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 498: ./mnist_convnet_modellearningrate_0.1\model.ckpt-498
EVAL EPOCH... 498
{'accuracy': 0.9712, 'loss': 0.10519544, 'global_step': 498}
INFO:tensorflow:Calling model_fn.
x_input.shape (128, 28, 28, 1)
convlayer1.shape (128, 28, 28, 16)
maxpool1.shape (128, 14, 14, 16)
convlayer2.shape (128, 14, 14, 16)
maxpool2.shape (128, 7, 7, 16)
maxpool2_vec.shape (128, 784)
denselayer.shape (128, 128)
logitslayer.shape (128, 10)
labels.shape (128,)
sceloss.shape ()
tf.estimator.ModeKeys.TRAIN...
INFO:te